In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
import math

In [3]:
data=input_data.read_data_sets("data/MNIST/",one_hot=True) 
data.test.cls = tf.argmax(data.test.labels, axis=1)
#one_hot= Whether to make the labels into a binary array
print("Size of:")
print("Training set \t\t{}".format(len(data.train.labels)))
print("Test set\t\t{}".format(len(data.test.labels)))
print("Validation set\t\t{}".format(len(data.validation.labels)))

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Size of:
Training set 		55000
Test set		10000
Validation set		5000


In [4]:
imgSize=28
imgSizeFlat=imgSize*imgSize
imgShape=(imgSize,imgSize)
noClasses=10
noColorChannels=1
batchSize=100
logs_path = "./logs/03/visualize_graph"

In [5]:
xFlat=tf.placeholder(tf.float32,shape=[None,imgSizeFlat],name='xFlat')
x2D=tf.reshape(xFlat,[-1,imgSize,imgSize,noColorChannels])
yTrue=tf.placeholder(tf.float32,shape=[None,noClasses],name='yTrue')
yTrueClass=tf.argmax(yTrue,axis=1)

In [8]:
def convLayer2D(inputTensor,kernelShape,noKernels):
    layer1_filterLength=kernelShape[0]
    layer1_filterHeight=kernelShape[1]
    layer1_noFilters=noKernels
    noChannels=tf.shape(inputTensor)[4]

    layer1_shape=[layer1_filterLength,layer1_filterHeight,noChannels,layer1_noFilters]
    layer1_weights=tf.Variable(tf.truncated_normal(layer1_shape,stddev=0.5))
    layer1_biases=tf.Variable(tf.constant(0.05,shape=[layer1_noFilters]))
    layer1_out=tf.nn.conv2d(input=x2D,filter=layer1_weights,strides=[1,1,1,1],padding='SAME')
    #strides=[img,x,y,colourChannel]
    layer1_out=layer1_out+layer1_biases
    
    return layer1_out




In [9]:
def maxPoolingLayer(inputTensor,kernelSize):
    return tf.nn.max_pool(value=inputTensor,ksize=[1,kernelSize[0],kernelSize[1],1],strides=[1,kernelSize[0],kernelSize[1],1],padding='SAME')


In [10]:
def relu(inputTensor):    
    return tf.nn.relu(layer1_out)

In [11]:
def flatten(inputTensor):
    layer3_shape=inputTensor.get_shape()  # layer_shape == [num_images, img_height, img_width, num_channels]
    layer3_noFeatures=int(layer3_shape[1]*layer3_shape[2]*layer3_shape[3])
    layer3_out=tf.reshape(inputTensor,[-1,layer3_noFeatures])
    return layer3_out

In [13]:
def fullyConnected(inputTensor,outputSize):
    layer4_outSize=outputSize
    layer4_shape=[layer3_noFeatures,layer4_outSize]
    layer4_weights=tf.Variable(tf.truncated_normal(layer4_shape,stddev=0.5))
    layer4_biases=tf.Variable(tf.constant(0.05,shape=[layer4_outSize]))
    layer4_out=tf.matmul(layer3_out,layer4_weights)+layer4_biases
    return layer4_out